In [172]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:150% !important; }</style>"))

# PROGRAM LAYOUT FROM IXSFIT

In [173]:
#!/usr/bin/env python
import os
import sys
import glob
import socket
import smtplib
import datetime
import subprocess
import itertools
import threading
import time
import sys
import webbrowser
import numpy as np
import math
import os
import tempfile
os.environ["MPLCONFIGDIR"] = tempfile.gettempdir()
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import tkinter as tk
from tkinter import BOTH, END, LEFT
from tkinter import *
import subprocess as sub
from tkinter import filedialog
import webbrowser
import ufit
import numpy as np
from ufit.lab import *
#import tkMessageBox
#os.environ[ 'MPLCONFIGDIR' ] = 'D:\.matplotlib'

#Main master GUI properties
specle = tk.Tk()
specle.title('HB1 ToolBox 2024 (c) Avishek Maity')
#master.geometry('300x700+10+10')

set_datatemplate('C:/Users/num/Documents/cycle 506/exp932/Datafiles/HB1_exp0932_scan%04d.dat')
ufit.set_dataformat('simple')


global xye_file
xye_file='X:/path/to/spec_file.dat'

n=0
#tk.Label(spec, text="SUPPLEM", font='TkFixedFont', width=12, anchor='w').grid(row=n+0, column=0);  
e0_1 = tk.Entry(specle, width=37); e0_1.insert(END,xye_file);  e0_1.grid(row=n+1, column=0, columnspan=2, pady=0)
#tk.Label(spec, text="SUMMARY", font='TkFixedFont', width=12, anchor='w').grid(row=n+1, column=0);      

def UploadSupplement():
    global spec_file, fname
    spec_file = filedialog.askopenfilename()
    for i in range(len(spec_file)):
        if spec_file[i]=='/':
            titl=spec_file[i+1:]
    n=0
    e0_1 = tk.Entry(specle, width=37); e0_1.insert(END,titl); e0_1.grid(row=n+1, column=0,columnspan=2,pady=0)
    #xye_file=str(e0_1.get())
    try:
        os.mkdir('./extracted')
    except:
        pass
    fout=open('./extracted/extractor.ini','w')
    
    fname=spec_file
    webbrowser.open(fname)
def MakeSummary():
    global fname
    #print(fname)
    fout0=open('./Summary.txt','w')
    f =open(fname, 'r')
    for line in f:
        if '#S' in line:
            linen=next(f)
            linen=linen.replace("\n","    ")
            fout0.write(linen + ':' + line)
    fout0.close()
    webbrowser.open('Summary.txt')
    
def scan2pol():
    scn_Pzz,scn_Pzx,scn_Pzy,scn_Pxx,scn_Pxy,scn_Pyy,scn_Pyx,scn_Pxz,scn_Pyz=eval()
    plabel=['Pzz','Pzx','Pzy','Pxx','Pxy','Pyy','Pyx','Pxz','Pyz']
    P = np.full((3,3), np.nan)
    u=[]
    d=[]
    u1=[]
    d1=[]
    i=0
    for n in [scn_Pzz,scn_Pzx,scn_Pzy,scn_Pxx,scn_Pxy,scn_Pyy,scn_Pyx,scn_Pxz,scn_Pyz]:    
        try:
            data=read_data(n)
            plt.plot(data['col_ds_nut'],55555*data['col_detector']/data['col_monitor'],'^-', label='scan'+str(n)+':'+plabel[i])
            
            u.append(data['col_detector'][0]/data['col_monitor'][0])
            d.append(data['col_detector'][1]/data['col_monitor'][1])
    
            u1.append(data['col_detector'][0])
            d1.append(data['col_detector'][1])
            i=i+1
            #print(n)
        except:
            pass
            print(n)
    plt.xlabel('SF                                                  ds_nut_amps (A)                                              NSF')
    plt.xticks([-3.2,3.2])
    plt.ylabel('counts / 55555 mon [~ 1 min]')
    plt.legend(fontsize=9, ncol=3)
    plt.title(data['subtitle'][:-4]+', T ='+str('%6.2f'%data['sample'])+' K'+', time ='+str('%6.2f'%data['time'])+' sec')
    up=np.zeros([3,3])
    dn=np.zeros([3,3])
      
    up[2,2],up[2,0],up[2,1],up[0,0],up[0,1],up[1,1],up[1,0],up[0,2],up[1,2]=u[0],u[1],u[2],u[3],u[4],u[5],u[6],u[7],u[8]
    dn[2,2],dn[2,0],dn[2,1],dn[0,0],dn[0,1],dn[1,1],dn[1,0],dn[0,2],dn[1,2]=d[0],d[1],d[2],d[3],d[4],d[5],d[6],d[7],d[8]
    
    up1=np.zeros([3,3])
    dn1=np.zeros([3,3])
       
    up1[2,2],up1[2,0],up1[2,1],up1[0,0],up1[0,1],up1[1,1],up1[1,0],up1[0,2],up1[1,2]=u1[0],u1[1],u1[2],u1[3],u1[4],u1[5],u1[6],u1[7],u1[8]
    dn1[2,2],dn1[2,0],dn1[2,1],dn1[0,0],dn1[0,1],dn1[1,1],dn1[1,0],dn1[0,2],dn1[1,2]=d1[0],d1[1],d1[2],d1[3],d1[4],d1[5],d1[6],d1[7],d1[8]
    
    P=(up1-dn1)/(up1+dn1)
    
    txt_up='NSF=\n'\
     '['+str('%5d'%up1[0,0])+', '+str('%5d'%up1[0,1])+', '+str('%5d'%up1[0,2])+']'+'\n'\
    +'['+str('%5d'%up1[1,0])+', '+str('%5d'%up1[1,1])+', '+str('%5d'%up1[1,2])+']'+'\n'\
    +'['+str('%5d'%up1[2,0])+', '+str('%5d'%up1[2,1])+', '+str('%5d'%up1[2,2])+']'
    
    txt_dn='SF=\n'\
     '['+str('%5d'%dn1[0,0])+', '+str('%5d'%dn1[0,1])+', '+str('%5d'%dn1[0,2])+']'+'\n'\
    +'['+str('%5d'%dn1[1,0])+', '+str('%5d'%dn1[1,1])+', '+str('%5d'%dn1[1,2])+']'+'\n'\
    +'['+str('%5d'%dn1[2,0])+', '+str('%5d'%dn1[2,1])+', '+str('%5d'%dn1[2,2])+']'

    txt_P='P=\n'\
     '['+str('%+5.4f'%P[0,0])+', '+str('%+5.4f'%P[0,1])+', '+str('%+5.4f'%P[0,2])+']'+'\n'\
    +'['+str('%+5.4f'%P[1,0])+', '+str('%+5.4f'%P[1,1])+', '+str('%+5.4f'%P[1,2])+']'+'\n'\
    +'['+str('%+5.4f'%P[2,0])+', '+str('%+5.4f'%P[2,1])+', '+str('%+5.4f'%P[2,2])+']'
    
    
    plt.text(-3.2, 0.65*55555*np.max(u), txt_up, fontsize=9, color='blue')
    plt.text(-1.2, 0.65*55555*np.max(u), txt_dn, fontsize=9, color='red')
    plt.text(0, 0.25*55555*np.max(u), txt_P, fontsize=9, color='green')
    
    figname=data['subtitle'][:-4]+'_T'+str(int(data['sample']))+'K'+'count'+str(int(data['time']))+'sec.jpg'
    
    print('saved as:',figname)
    plt.savefig(figname, dpi=600)
    plt.show()

def ScanExtract():
    global fname, rawscanlist
    rawscanlist=[]
    f =open(fname, 'r')
    try:
        os.mkdir('./extracted')
    except:
        pass
    fout=open('./extracted/extractor.ini','w')
    for line in f:
        if '#S' in line:
            rawscanlist.append('./extracted/scan_'+str(int(line[2:6]))+'.dat')
            fout=open('./extracted/scan_'+str(int(line[2:6]))+'.dat','w')
            fout.write(line)
        if '#D' in line:
            fout.write(line)
        if '#L' in line:
            fout.write(line)
        if '#' not in line:
            fout.write(line)    
def PlotRawData():
    UploadScan()
    for i in range(len(scan)):
        if scan[i]=='/':
            titl=scan[i+1:]
    mon,p=eval(e0_2.get())
        
    
    d=np.loadtxt(scan)
    plt.rcParams['figure.figsize'] = [9, 4.5]
    plt.rcParams.update({'font.size': 12})
    plt.rcParams['axes.linewidth'] = 1
    plt.rcParams["legend.markerscale"] = 1
    import matplotlib as mpl
    mpl.rcParams['patch.linewidth'] = 0
    from matplotlib.ticker import AutoMinorLocator
    fig, ax1 = plt.subplots(1, 1, sharex=False, sharey=False,squeeze=True)
    ax1.tick_params(axis="y",direction="in", width=1.5)
    ax1.tick_params(axis="x",direction="in", width=1.5)
    ax1.tick_params(bottom=True, top=True, left=True, right=True,\
                labelbottom=True, labeltop=False, labelleft=True, labelright=False)
    ax1.xaxis.set_minor_locator(AutoMinorLocator())
    ax1.yaxis.set_minor_locator(AutoMinorLocator())
    ax1.tick_params(axis="x", which="minor", direction="in", width=1,
                      top=True, labeltop=False, bottom=True, labelbottom=False)
    ax1.tick_params(axis="y", which="minor", direction="in", width=1,
                      right=True, labelright=False, left=True, labelleft=False)
    plt.title(titl)
    if len(p)==3:
        x,y,e=d[:,int(p[0])], d[:,int(p[1])]*(mon/d[:,int(p[2])]), np.sqrt(d[:,int(p[1])])*(mon/d[:,int(p[2])])
        ax1.errorbar(x,y,xerr=None, yerr=e, marker='o', color='r',\
                 ms=5, mfc='none', linewidth=1., elinewidth=1,\
                 capsize=1, capthick=1)
    plt.legend()
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.ylabel('Y')
    plt.xlabel('X')
    plt.show()
def SeparateAna():
    global rawscanlist, rawscanlistana
    rawscanlistana=[]
    for i in range(len(rawscanlist)):
        fin=open(rawscanlist[i], 'r')
        d=np.loadtxt(rawscanlist[i])
        stype, xcol,ycol,mon,analyzer= eval(e1_1.get())
        if analyzer==1 or analyzer==2 or analyzer==1  or analyzer==3  or analyzer==4  or analyzer==5  or analyzer==6  or analyzer==7  or analyzer==8 or analyzer==9:
            if stype=='hklscan':
                for line in fin:
                    if '#S' and stype in line:
                        fout=open(rawscanlist[i][:-4]+'_ana_'+str(analyzer)+'.dat','w')
                        rawscanlistana.append(rawscanlist[i][:-4]+'_ana_'+str(analyzer)+'.dat')
                        fout.write(line)    
                        for i in range(len(d)):
                            dataline=str('%.10e'%d[i,xcol])+'\t'+ str('%.10e'%d[i, ycol]) +'\t'+ str('%.10e'%d[i, mon])
                            fout.write(dataline+'\n')

def NormalizeScans():
    global rawscanlist, rawscanlistana, rawscanlistananorm
    rawscanlistananorm=[]
    for i in range(len(rawscanlistana)):
        fin=open(rawscanlistana[i], 'r')
        d=np.loadtxt(rawscanlistana[i])
        fout=open(rawscanlistana[i][:-4]+'_norm.dat','w')
        rawscanlistananorm.append(rawscanlistana[i][:-4]+'_norm.dat')
        for line in fin:
            if '#S' in line:
                fout.write(line)
            if '#L' in line:
                fout.write(line)
        fout.write('#NORMALIZED BY MONITOR:'+str(eval(e2_1.get()))+'\n')
        for i in range(len(d)):
            norm=eval(e2_1.get())/d[i, 2]
            dataline=str('%.10e'%d[i,0])+'\t'+ str('%.10e'%(d[i, 1]*norm)) +'\t'+ str('%.10e'%(d[i, 2]*norm))
            fout.write(dataline+'\n')
def CorrectE0line():
    from lmfit.models import PseudoVoigtModel
    from scipy.signal import find_peaks
    scan=filedialog.askopenfilename()
    for i in range(len(scan)):
        if scan[i]=='/':
            titl=scan[i+1:]
    d=np.loadtxt(scan)
    el,er, lfp, rfp, pwdth, choice=eval(e3_1.get())
    x,y,e=d[:,0], d[:,1], np.sqrt( d[:,1])
    p,prop=find_peaks(y, distance=None , width=pwdth)
    cen0,wid0, hight0=0,0,0
    peak0=[cen0,wid0, hight0]
    peak1=[cen0,wid0, hight0]
    peak2=[cen0,wid0, hight0]
    peak3=[cen0,wid0, hight0]
    for i in range(len(p)):
        if  el<x[p[i]]<er:
            cen0=x[p[i]]
            xmin=x[prop["left_ips"][i].astype(int)]
            xmax=x[prop["right_ips"][i].astype(int)]
            wid0=xmax-xmin
            hight0=prop["prominences"][i]
            #height=prop["prominences"][pos]
            peak0=[cen0,wid0, hight0]
            n=p[i]
        elif 2.5<x[p[i]]<7.:
            cen0=x[p[i]]
            xmin=x[prop["left_ips"][i].astype(int)]
            xmax=x[prop["right_ips"][i].astype(int)]
            wid0=xmax-xmin
            hight0=prop["prominences"][i]
            #height=prop["prominences"][pos]
            peak1=[cen0,wid0, hight0]
        elif 7<x[p[i]]<11.5:
            cen0=x[p[i]]
            xmin=x[prop["left_ips"][i].astype(int)]
            xmax=x[prop["right_ips"][i].astype(int)]
            wid0=xmax-xmin
            hight0=prop["prominences"][i]
            #height=prop["prominences"][pos]
            peak2=[cen0,wid0, hight0]
        elif 11.5<x[p[i]]<15.5:
            cen0=x[p[i]]
            xmin=x[prop["left_ips"][i].astype(int)]
            xmax=x[prop["right_ips"][i].astype(int)]
            wid0=xmax-xmin
            hight0=prop["prominences"][i]
            #height=prop["prominences"][pos]
            peak3=[cen0,wid0, hight0]
            
    pvmodel=PseudoVoigtModel(prefix='pv1_')
    pvmodel.nan_policy='omit'
    pvmodel.set_param_hint('pv1_amplitude', vary=True,  value=peak0[1])#, min=0, max=5.0e-4)#%
    pvmodel.set_param_hint('pv1_center',    vary=True,  value=peak0[0])#,  min=-2.5, max=2.5)#%
    pvmodel.set_param_hint('pv1_fwhm',      vary=True,  value=peak0[2])#,   min=0.1, max=2)#%
    pvmodel.set_param_hint('pv1_fraction',  vary=True,  value=0.5)#,      min=0, max=1)#%
    result = pvmodel.fit(y[n-lfp:n+rfp],x=x[n-lfp:n+rfp])
    peak0[0]=result.params['pv1_center'].value
    peak0[1]=result.params['pv1_fwhm'].value
    peak0[2]=result.params['pv1_height'].value
    if peak1[0] != 0 and  peak1[1]!=0 and peak1[2]!=0:
        peak1[0]=peak1[0]-peak0[0]
    if peak2[0] != 0 and  peak2[1]!=0 and peak2[2]!=0:
        peak2[0]=peak2[0]-peak0[0]

        
    plt.rcParams['figure.figsize'] = [9, 4.5]
    plt.rcParams.update({'font.size': 12})
    plt.rcParams['axes.linewidth'] = 1
    plt.rcParams["legend.markerscale"] = 1
    import matplotlib as mpl
    mpl.rcParams['patch.linewidth'] = 0
    from matplotlib.ticker import AutoMinorLocator
    
    fig, ax1 = plt.subplots(1, 1, sharex=False, sharey=False,squeeze=True)
    ax1.tick_params(axis="y",direction="in", width=1.5)
    ax1.tick_params(axis="x",direction="in", width=1.5)
    ax1.tick_params(bottom=True, top=True, left=True, right=True,\
                labelbottom=True, labeltop=False, labelleft=True, labelright=False)
    ax1.xaxis.set_minor_locator(AutoMinorLocator())
    ax1.yaxis.set_minor_locator(AutoMinorLocator())
    ax1.tick_params(axis="x", which="minor", direction="in", width=1,
                      top=True, labeltop=False, bottom=True, labelbottom=False)
    ax1.tick_params(axis="y", which="minor", direction="in", width=1,
                      right=True, labelright=False, left=True, labelleft=False)    
      
    plt.title(titl)
    plt.plot(x,y,'--', label='raw data') 
    plt.plot(x[p], y[p], "k*", ms=10, label='possible peaks')
    plt.hlines(y=prop["width_heights"], xmin=x[prop["left_ips"].astype(int)],
               xmax=x[prop["right_ips"].astype(int)], color = "b")
    plt.vlines(x=x[p], ymin=y[p]-prop["prominences"],ymax=y[p], color = "b")
    plt.errorbar(x[n-lfp:n+rfp],y[n-lfp:n+rfp],yerr=e[n-lfp:n+rfp], xerr=None, marker='o', mfc=None,label='data to fit')
    plt.plot(x[n-lfp:n+rfp], result.best_fit, 'k-', lw=1, label='pv fit')
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.ylabel('Normalized counts')
    plt.xlabel('E (meV)')
    plt.show()   
    
    peaks=[peak0, peak1, peak2, peak3]
    d[:,0]=x-peak0[0]
    if choice==True:
        fout=open(scan[:-4]+'_e0.dat','w')
        fin=open(scan,'r')
        for line in fin:
            if '#' in line:
                fout.write(line)
        np.savetxt(fout,\
               d, fmt='%.10e', delimiter='\t', newline='\n',\
               header='', footer='', comments='# ', encoding=None)
        fout.close()
def UploadScan():
    global scan
    scan=filedialog.askopenfilename()  
def PlotXYEData():
    #global scan, stddev, titl
    UploadScan()
    for i in range(len(scan)):
        if scan[i]=='/':
            titl=scan[i+1:]
    d=np.loadtxt(scan)
    plt.rcParams['figure.figsize'] = [9, 4.5]
    plt.rcParams.update({'font.size': 12})
    plt.rcParams['axes.linewidth'] = 1
    plt.rcParams["legend.markerscale"] = 1
    import matplotlib as mpl
    mpl.rcParams['patch.linewidth'] = 0
    from matplotlib.ticker import AutoMinorLocator

    fig, ax1 = plt.subplots(1, 1, sharex=False, sharey=False,squeeze=True)
    ax1.tick_params(axis="y",direction="in", width=1.5)
    ax1.tick_params(axis="x",direction="in", width=1.5)
    ax1.tick_params(bottom=True, top=True, left=True, right=True,\
                labelbottom=True, labeltop=False, labelleft=True, labelright=False)
    ax1.xaxis.set_minor_locator(AutoMinorLocator())
    ax1.yaxis.set_minor_locator(AutoMinorLocator())
    ax1.tick_params(axis="x", which="minor", direction="in", width=1,
                      top=True, labeltop=False, bottom=True, labelbottom=False)
    ax1.tick_params(axis="y", which="minor", direction="in", width=1,
                      right=True, labelright=False, left=True, labelleft=False)
    
    p=eval(e4_1.get())
    legen=['energy', 'sum0', 'std3', 'std5']

    plt.title(titl) 
    
    if len(p)==1:
        x,y,e=d[:,0], d[:,int(p[0])], np.sqrt(d[:,int(p[0])])
        ax1.errorbar(x,y,xerr=None, yerr=e, marker='o', color='r',\
                 ms=5, mfc='none', linewidth=1., elinewidth=1,\
                 capsize=1, capthick=1, label=legen[int(p[0])])

    if len(p)==2:
        x,y,e=d[:,0], d[:,int(p[0])], np.sqrt(d[:,int(p[0])])
        ax1.errorbar(x,y,xerr=None, yerr=e, marker='o', color='r',\
                 ms=5, mfc='none', linewidth=1., elinewidth=1,\
                 capsize=1, capthick=1, label=legen[int(p[0])])
        x,y,e=d[:,0], d[:,int(p[1])], np.sqrt(d[:,int(p[1])])
        ax1.errorbar(x,y,xerr=None, yerr=e, marker='o', color='b',\
                 ms=5, mfc='none', linewidth=1., elinewidth=1,\
                 capsize=1, capthick=1, label=legen[int(p[1])])
        
    if len(p)==3:
        x,y,e=d[:,0], d[:,int(p[0])], np.sqrt(d[:,int(p[0])])
        ax1.errorbar(x,y,xerr=None, yerr=e, marker='o', color='r',\
                 ms=5, mfc='none', linewidth=1., elinewidth=1,\
                 capsize=1, capthick=1, label=legen[int(p[0])])
        x,y,e=d[:,0], d[:,int(p[1])], np.sqrt(d[:,int(p[1])])
        ax1.errorbar(x,y,xerr=None, yerr=e, marker='o', color='b',\
                 ms=5, mfc='none', linewidth=1., elinewidth=1,\
                 capsize=1, capthick=1, label=legen[int(p[1])])
        x,y,e=d[:,0], d[:,int(p[2])], np.sqrt(d[:,int(p[2])])
        ax1.errorbar(x,y,xerr=None, yerr=e, marker='o', color='g',\
                 ms=5, mfc='none', linewidth=1., elinewidth=1,\
                 capsize=1, capthick=1, label=legen[int(p[2])])
        
    plt.legend()
    plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
    plt.ylabel('Normalized counts')
    plt.xlabel('E (meV)')
    plt.show()  
def RemoveSpikes():

    scan=filedialog.askopenfilename()
    for i in range(len(scan)):
        if scan[i]=='/':
            titl=scan[i+1:]
    d=np.loadtxt(scan)
    threshold,threshold1,choice=eval(e5_1.get())
    d1=(d[:,1].max()-d[:,1].min())*10e-8
    d2=(d[:,2].max()-d[:,2].min())*10e-8
    d3=(d[:,3].max()-d[:,3].min())*10e-8
    
    for i in range(len(d)-2):
        try:
            #r1=(d[i+1,1]/d[i,1])/(d[i+2,1]/d[i+1,1])
            r1=((d[i+1,1]+d1)/(d[i,1]+d1))/((d[i+2,1]+d1)/(d[i+1,1]+d1))
            if r1>threshold and r1<threshold1:
                d[i+1,0],d[i+1,1]=None,None
        except:
            pass        
        try:            
            #r2=(d[i+1,2]/d[i,2])/(d[i+2,2]/d[i+1,2])
            r2=((d[i+1,2]+d2)/(d[i,2]+d2))/((d[i+2,2]+d2)/(d[i+1,2]+d2))
            if r2>threshold and r1<threshold1:
                d[i+1,0],d[i+1,2]=None,None    
        except:
            pass        
        try:            
            #r3=(d[i+1,3]/d[i,3])/(d[i+2,3]/d[i+1,3])
            r3=((d[i+1,3]+d3)/(d[i,3]+d3))/((d[i+2,3]+d3)/(d[i+1,3]+d3))
            if r3>threshold and r1<threshold1:
                d[i+1,0],d[i+1,3]=None,None                 
        except:
            pass            
    if choice==True:
        fout=open(scan[:-4]+'_rsp.dat','w')
        fin=open(scan,'r')
        for line in fin:
            if '#' in line:
                fout.write(line)
        np.savetxt(fout,\
               d, fmt='%.18e', delimiter='\t', newline='\n',\
               header='', footer='', comments='# ', encoding=None)
        fout.close()
        
    p=[1,2,3] 
    legen=['energy', 'sum0', 'std3', 'std5']
    plt.rcParams['figure.figsize'] = [9, 4.5]
    plt.rcParams.update({'font.size': 12})
    plt.rcParams['axes.linewidth'] = 1
    plt.rcParams["legend.markerscale"] = 1
    import matplotlib as mpl
    mpl.rcParams['patch.linewidth'] = 0
    from matplotlib.ticker import AutoMinorLocator

    fig, ax1 = plt.subplots(1, 1, sharex=False, sharey=False,squeeze=True)
    ax1.tick_params(axis="y",direction="in", width=1.5)
    ax1.tick_params(axis="x",direction="in", width=1.5)
    ax1.tick_params(bottom=True, top=True, left=True, right=True,\
                labelbottom=True, labeltop=False, labelleft=True, labelright=False)
    ax1.xaxis.set_minor_locator(AutoMinorLocator())
    ax1.yaxis.set_minor_locator(AutoMinorLocator())
    ax1.tick_params(axis="x", which="minor", direction="in", width=1,
                      top=True, labeltop=False, bottom=True, labelbottom=False)
    ax1.tick_params(axis="y", which="minor", direction="in", width=1,
                      right=True, labelright=False, left=True, labelleft=False)    
    plt.title(titl)
    if len(p)==3:
        x,y,e=d[:,0], d[:,int(p[0])], np.sqrt(d[:,int(p[0])])
        ax1.errorbar(x,y,xerr=None, yerr=e, marker='o', color='r',\
                 ms=5, mfc='none', linewidth=1., elinewidth=1,\
                 capsize=1, capthick=1, label=legen[int(p[0])])
        x,y,e=d[:,0], d[:,int(p[1])], np.sqrt(d[:,int(p[1])])
        ax1.errorbar(x,y,xerr=None, yerr=e, marker='o', color='b',\
                 ms=5, mfc='none', linewidth=1., elinewidth=1,\
                 capsize=1, capthick=1, label=legen[int(p[1])])
        x,y,e=d[:,0], d[:,int(p[2])], np.sqrt(d[:,int(p[2])])
        ax1.errorbar(x,y,xerr=None, yerr=e, marker='o', color='g',\
                 ms=5, mfc='none', linewidth=1., elinewidth=1,\
                 capsize=1, capthick=1, label=legen[int(p[2])])
        plt.legend()
        plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
        plt.ylabel('Normalized counts')
        plt.xlabel('E (meV)')
        plt.show()    
        
n=1
tk.Button(specle,text='LOAD',command=UploadSupplement, bg='pink', font='Times 10').grid(row=n+0, column=2, rowspan=1, columnspan=1, pady=0, padx=0)

tk.Button(specle,text='MAKE SUMMARY',command=MakeSummary,  bg='pink', font='Times 10').grid(row=n+1, column=0, rowspan=1, columnspan=1, pady=0, padx=0)
tk.Button(specle,text='EXTRACT SCANS',command=ScanExtract, bg='pink', font='Times 10').grid(row=n+1, column=1, rowspan=1, columnspan=1, pady=0, padx=0)
tk.Button(specle,text='PLOT RAWDATA  ',command=PlotRawData, bg='pink', font='Times 10').grid(row=n+4, column=0, rowspan=1, columnspan=1, pady=0, padx=0)
e0_2 = tk.Entry(specle, width=17); e0_2.insert(END,'1000000,[0,41,47]');  e0_2.grid(row=n+4, column=1, columnspan=1, pady=0)
n=5
tk.Button(specle,text=' SEPARATE ANA   ',command=SeparateAna, bg='pink', font='Times 10').grid(row=n+3, column=0, rowspan=1, columnspan=1, pady=0, padx=0)
e1_1 = tk.Entry(specle, width=17); e1_1.insert(END,'["hklscan",0,41,47,6]');  e1_1.grid(row=n+3, column=1, columnspan=1, pady=0)

n=9
tk.Button(specle,text='NORMALIZATION',command=NormalizeScans,  bg='pink', font='Times 10').grid(row=n+2, column=0, rowspan=1, columnspan=1, pady=0, padx=0)
e2_1 = tk.Entry(specle, width=17); e2_1.insert(END,'1000000');  e2_1.grid(row=n+2, column=1, columnspan=1, pady=0)

tk.Button(specle,text='CORRECT E0-LINE ',command=CorrectE0line, bg='pink', font='Times 10').grid(row=n+3, column=0, rowspan=1, columnspan=1, pady=0, padx=0)
e3_1 = tk.Entry(specle, width=17); e3_1.insert(END,'-1,1,5,5,2,True');  e3_1.grid(row=n+3, column=1, columnspan=1, pady=0)



tk.Button(specle,text='PLOT XYE DATA  ',command=PlotXYEData, bg='pink', font='Times 10').grid(row=n+4, column=0, rowspan=1, columnspan=1, pady=0, padx=0)
e4_1 = tk.Entry(specle, width=17); e4_1.insert(END,'[1,2,3]');  e4_1.grid(row=n+4, column=1, columnspan=1, pady=0)

tk.Button(specle,text='  REMOVE SPIKES  ',command=RemoveSpikes, bg='pink', font='Times 10').grid(row=n+5, column=0, rowspan=1, columnspan=1, pady=0, padx=0)
e5_1 = tk.Entry(specle, width=17); e5_1.insert(END,'5, np.inf, True');  e5_1.grid(row=n+5, column=1, columnspan=1, pady=0)

n=80
tk.Button(specle, text='QUIT',command=specle.destroy, font='Times 10').grid(row=n+0, column=2, rowspan=1, columnspan=1,  pady=0)
#tk.mainloop()


ImportError: Cannot load backend 'TkAgg' which requires the 'tk' interactive framework, as 'qt' is currently running

# HB1_ToolBox

In [48]:
#!/usr/bin/env python
import os
import sys
import glob
import socket
import smtplib
import datetime
import subprocess
import itertools
import threading
import time
import sys
import webbrowser
import numpy as np
import math
import os
import tempfile
os.environ["MPLCONFIGDIR"] = tempfile.gettempdir()
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import tkinter as tk
from tkinter import BOTH, END, LEFT
from tkinter import *
import subprocess as sub
from tkinter import filedialog
import webbrowser
import ufit
import numpy as np
from ufit.lab import *
#import tkMessageBox
#os.environ[ 'MPLCONFIGDIR' ] = 'D:\.matplotlib'

#Main master GUI properties
ptax = tk.Tk()
ptax.title('HB1 ToolBox 2024 (c) Avishek Maity')
#master.geometry('300x700+10+10')

set_datatemplate('C:/Users/num/Documents/cycle 506/exp932/Datafiles/HB1_exp0932_scan%04d.dat')
ufit.set_dataformat('simple')


global hb1_file
hb1_file='X:/path/to/HB1_data_file.dat'
# Define Our Images
                 
n=0
m=0
e0_1 = tk.Entry(ptax, width=37); e0_1.insert(END,hb1_file);  e0_1.grid(row=n+1, column=m+0, columnspan=2, pady=0)
e0_2 = tk.Entry(ptax, width=37); e0_2.insert(END,hb1_file);  e0_2.grid(row=n+2, column=m+0, columnspan=2, pady=0)
def load():
    global ptax_file, fname, n, m
    ptax_file = filedialog.askopenfilename()
    for i in range(len(spec_file)):
        if ptax_file[i]=='/':
            titl=ptax_file[i+1:]
    e0_1 = tk.Entry(ptax, width=37); e0_1.insert(END,titl); e0_1.grid(row=n+1, column=m+0,columnspan=2,pady=0)
    

def view():
    fname=ptax_file
    webbrowser.open(fname)

tk.Button(ptax,text='LOAD',command=load,  bg='pink', font='Times 10').grid(row=n+1, column=m+2, rowspan=1, columnspan=1, pady=0, padx=0)
tk.Button(ptax,text='VIEW',command=view,  bg='pink', font='Times 10').grid(row=n+1, column=m+3, rowspan=1, columnspan=1, pady=0, padx=0)
n=80
tk.Button(ptax, text='QUIT',command=ptax.destroy, font='Times 10').grid(row=n+0, column=2, rowspan=1, columnspan=1,  pady=0)
#tk.mainloop()

# NON GUI VERSION

In [58]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:150% !important; }</style>"))

import ufit
import numpy as np
from ufit.lab import *

DIR='C:/Users/num/Documents/cycle506/exp944/Datafiles/'
FIL_TEMPLATE='HB1_exp0944_scan%04d.dat'
DATA_TEMPLATE=DIR+FIL_TEMPLATE

# UFIT PACKAGE
set_datatemplate('C:/Users/num/Documents/cycle506/exp944/Datafiles/HB1_exp0944_scan%04d.dat')
ufit.set_dataformat('taipan')
d=read_data(51)
d['meta']


{'instrument': 'TAIPAN',
 'filenumber': 51,
 'date': '6/19/2024',
 'time': 30.0,
 'proposal': '32296',
 'experiment': '944',
 'experiment_number': '944',
 'command': 'scan ds_nut 3.2 -3.2 6.4',
 'builtin_command': 'scan ds_nut 3.2 -3.2 6.4',
 'users': 'Arnab Banerjee, Bishnu Prasad Belbase, Colten Koogler, Fankang Li, Gavin Hester, Guga Khundzakishvili, Kevin Goodman, Neel Jain, Chenyang Jiang, Robert Cooper, Jacob Tosado, Yong Chen',
 'local_contact': 'Masa Matsuda',
 'subtitle': 'Bi2Se3_Feanor T = 5K I = -100mA (000) no_chopper Pyy',
 'monochromator': 'Heusler',
 'analyzer': 'Heusler',
 'sense': '+-+',
 'collimation': '48-80-60-999',
 'samplename': 'Bismuth Selenide Bi2Se3 Feanor',
 'sampletype': 'crystal',
 'samplemosaic': '30.000000',
 'latticeconstants': '4.130000,4.130000,28.560000,90.000000,90.000000,120.000000',
 'ubmatrix': '0.141570,0.273890,0.007001,-0.030032,-0.055866,0.034307,0.239219,-0.005717,0.000164',
 'mode': '0',
 'plane_normal': '0.019101,-0.008668,0.999780',
 'ubco

In [ ]:
import numpy as np




In [2]:
# MAKE SUMMARY
#f=open('summary.txt', 'w')
#plt.figure()
f=None
ufit.set_dataformat('taipan')
for n in range(0,1000):
    try:
            d=read_data(n)
        #if 'scan h 0 k 0 l' in '%50s'%d['command'] and d['hguide']>0.5 and d['vguide']>0.5 and d['tbguide']<0.5 and d['cguide']>17.5:
            if d['hguide']<0.5 and d['vguide']<0.5 and d['cguide']>17.5:
                print(TCYAN +'%3d'%n, '%50s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|','%10s'%'[NSF P-X] floff, guide -16 parq', file=f)
    
            elif d['hguide']<0.5 and d['vguide']<0.5 and d['cguide']<-17.5:
                print(TYELO +'%3d'%n, '%50s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|','%10s'%'[NSF PX] floff, guide 16 parq', file=f)        
    
            elif d['hguide']<0.5 and d['vguide']<0.5 and d['tbguide']>4.5:
                print(TBLACK + '%3d'%n, '%50s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|','%10s'%'[NSF PZ] floff, guide 18 perpq', file=f)
                
            elif d['hguide']>0.5 and d['vguide']>0.5 and d['tbguide']<0.5 and d['cguide']>17.5:
                print(TGREEN + '%3d'%n, '%50s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|','%22s'%'[SF P-X] flon, guide -16 parq', file=f)
                
            elif d['hguide']>0.5 and d['vguide']>0.5 and d['tbguide']<0.5 and d['cguide']<-17.5:
                print(TPURP + '%3d'%n, '%50s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|','%20s'%'[SF PX] flon, guide 16 parq', file=f)
    
            elif d['hguide']>0.5 and d['vguide']>0.5 and d['tbguide']<0.5 and 7<d['cguide']<9:
                print(TBLUE + '%3d'%n, '%50s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|','%22s'%'[SF PY] flon, guide 18 perpqh', file=f)
                    
            elif d['hguide']>0.5 and d['vguide']>0.5 and d['tbguide']>4.5:
                print(TRED + '%3d'%n, '%50s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|', '%21s'%'[SF PZ] flon, guide 18 perpq', file=f)


            #plt.errorbar(d['col_l'], 1000*d['col_detector']/d['col_monitor'],yerr=1000*np.sqrt(d['col_detector'])/d['col_monitor'], xerr=None,\
                        #marker='o', ms=5)

    except:
        pass
#plt.xlabel('(0,0,l) r.l.u')
#plt.ylabel('normalized counts')
#plt.show()

In [3]:

# MAKE SUMMARY
x,y,y1,y2,y3,y4,y5,y6=[],[],[],[],[],[],[],[]
ufit.set_dataformat('taipan')
for n in range(1000):
    try:
        x.append(n)
        d=read_data(n)
        y.append(d['fguide'])
        y1.append(d['hguide'])
        y2.append(d['vguide'])
        
        y3.append(d['tbguide'])
        y4.append(d['aguide'])
        y5.append(d['bguide'])
        y6.append(d['cguide'])
    except:
        pass

plt.figure()
plt.plot(x[:284],y ,'o', ms=1, label='fguide')
plt.plot(x[:284],y1,'o', ms=1, label='hguide')
plt.plot(x[:284],y2,'o', ms=1, label='vguide')
plt.plot(x[:284],y3,'o', ms=1, label='tbguide')
plt.plot(x[:284],y4,'o', ms=1, label='aguide')
plt.plot(x[:284],y5,'o', ms=1, label='bguide')
plt.plot(x[:284],y6,'o', ms=1, label='cguide')
plt.legend()
plt.show()

In [188]:
f.close()


In [59]:

TBLACK =  '\033[30m'
print (TBLACK + "It doens't reset!" , TBLACK)
TRED =  '\033[31m'
print (TRED + "It doens't reset!" , TRED)
TGREEN =  '\033[32m' 
print (TGREEN + "It doens't reset!" , TGREEN)
TYELO =  '\033[33m'
print (TYELO + "It doens't reset!" , TYELO)
TBLUE =  '\033[34m'
print (TBLUE + "It doens't reset!" , TBLUE)
TPURP =  '\033[35m'
print (TPURP + "It doens't reset!" , TPURP)
TCYAN =  '\033[36m'
print (TCYAN + "It doens't reset!" , TCYAN)


It doens't reset! 
It doens't reset! 
It doens't reset! 
It doens't reset! 
It doens't reset! 
It doens't reset! 
It doens't reset! 


In [60]:
#library
import ufit
import numpy as np
from ufit.lab import *

# data directory
DIR='C:/Users/num/Documents/cycle506/exp944/Datafiles/'
FIL_TEMPLATE='HB1_exp0944_scan%04d.dat'
DATA_TEMPLATE=DIR+FIL_TEMPLATE

# ufit data template
set_datatemplate('C:/Users/num/Documents/cycle506/exp945/Datafiles/HB1_exp0944_scan%04d.dat')
#ufit.set_dataformat('taipan')
f=None
ufit.set_dataformat('ptax')

# text colors
TBLACK,TRED,TGREEN,TYELO,TBLUE,TPURP,TCYAN =\
'\033[30m','\033[31m','\033[32m','\033[33m','\033[34m','\033[35m','\033[36m'


# scan ranges
for n in range(14,50):
    try:
            d=read_data(n)
            if d['hguide']<0.5 and d['vguide']<0.5 and d['cguide']>17.5:
                print(TCYAN +'%3d'%n, '|','%80s'%d['subtitle'], '|', '%55s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|','%10s'%'[NSF P-X] floff, guide -16 parq')
    
            elif d['hguide']<0.5 and d['vguide']<0.5 and d['cguide']<-17.5:
                print(TYELO +'%3d'%n, '|','%80s'%d['subtitle'], '|', '%55s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|','%10s'%'[NSF PX] floff, guide 16 parq')        
    
            elif d['hguide']<0.5 and d['vguide']<0.5 and d['tbguide']>4.5:
                print(TBLACK + '%3d'%n, '|','%80s'%d['subtitle'], '|', '%55s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|','%10s'%'[NSF PZ] floff, guide 18 perpq')
                
            elif d['hguide']>0.5 and d['vguide']>0.5 and d['tbguide']<0.5 and d['cguide']>17.5:
                print(TGREEN + '%3d'%n, '|','%80s'%d['subtitle'], '|', '%55s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|','%22s'%'[SF P-X] flon, guide -16 parq')
                
            elif d['hguide']>0.5 and d['vguide']>0.5 and d['tbguide']<0.5 and d['cguide']<-17.5:
                print(TPURP + '%3d'%n, '|','%80s'%d['subtitle'], '|', '%55s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|','%20s'%'[SF PX] flon, guide 16 parq')
    
            elif d['hguide']>0.5 and d['vguide']>0.5 and d['tbguide']<0.5 and 7<d['cguide']<9:
                print(TBLUE + '%3d'%n, '|','%80s'%d['subtitle'], '|', '%55s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|','%22s'%'[SF PY] flon, guide 18 perpqh')
                    
            elif d['hguide']>0.5 and d['vguide']>0.5 and d['tbguide']>4.5:
                print(TRED + '%3d'%n, '|','%80s'%d['subtitle'], '|', '%55s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|', '%21s'%'[SF PZ] flon, guide 18 perpq')

    except:
        pass


UFitError: Unknown data format: 'ptax', available formats are ill, nicos, old nicos, simple, simple comma-separated, trisp, llb, taipan, nist, desy, zebra, dynacool, cascade

In [218]:
d=read_data(196)

In [224]:
d['col_h']

array([-0.0001, -0.0002, -0.0002, -0.0001, -0.0001, -0.0001, -0.0001,
       -0.0001, -0.0001, -0.0001, -0.0001, -0.0002, -0.0001, -0.0001])

In [334]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:150% !important; }</style>"))

import ufit
import numpy as np
from ufit.lab import *
exp_no=936

DIR='C:/Users/num/Documents/cycle506/exp'+'%3s'%(exp_no)+'/Datafiles/'
FIL_TEMPLATE='HB1_exp0'+'%3s'%(exp_no)+'_scan%04d.dat'
DATA_TEMPLATE=DIR+FIL_TEMPLATE
f=None
# UFIT PACKAGE

set_datatemplate('C:/Users/num/Documents/cycle506/exp'+'%3s'%(exp_no)+'/Datafiles/HB1_exp0'+'%3s'%(exp_no)+'_scan%04d.dat')


for n in range(0,1000):
    try:
            d=read_data(n)
        #if 'scan h 0 k 0 l' in '%50s'%d['command'] and d['hguide']>0.5 and d['vguide']>0.5 and d['tbguide']<0.5 and d['cguide']>17.5:
            if d['hguide']<0.5 and d['vguide']<0.5 and d['cguide']>17.5:
                print(TCYAN +'%3d'%n, '%50s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|','%10s'%'[NSF P-X] floff, guide -16 parq', file=f)
    
            elif d['hguide']<0.5 and d['vguide']<0.5 and d['cguide']<-17.5:
                print(TYELO +'%3d'%n, '%50s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|','%10s'%'[NSF PX] floff, guide 16 parq', file=f)        
    
            elif d['hguide']<0.5 and d['vguide']<0.5 and d['tbguide']>4.5:
                print(TBLACK + '%3d'%n, '%50s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|','%10s'%'[NSF PZ] floff, guide 18 perpq', file=f)
                
            elif d['hguide']>0.5 and d['vguide']>0.5 and d['tbguide']<0.5 and d['cguide']>13.5:
                print(TGREEN + '%3d'%n, '%50s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|','%22s'%'[SF P-X] flon, guide -16 parq', file=f)
                
            elif d['hguide']>0.5 and d['vguide']>0.5 and d['tbguide']<0.5 and d['cguide']<-13.5:
                print(TPURP + '%3d'%n, '%50s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|','%20s'%'[SF PX] flon, guide 16 parq', file=f)
    
            elif d['hguide']>0.5 and d['vguide']>0.5 and d['tbguide']<0.5 and 7<d['cguide']<9:
                print(TBLUE + '%3d'%n, '%50s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|','%22s'%'[SF PY] flon, guide 18 perpqh', file=f)
                    
            elif d['hguide']>0.5 and d['vguide']>0.5 and d['tbguide']>4.5:
                print(TRED + '%3d'%n, '%50s'%d['command'],\
                      '|','%5.3f'%d['fguide'], '%7.3f'%d['hguide'], '%7.3f'%d['vguide'],\
                      '|', '%5.3f'%d['tbguide'], '%7.3f'%d['aguide'], '%7.3f'%d['bguide'], '%7.3f'%d['cguide'],\
                      '|', '%21s'%'[SF PZ] flon, guide 18 perpq', file=f)


            #plt.errorbar(d['col_l'], 1000*d['col_detector']/d['col_monitor'],yerr=1000*np.sqrt(d['col_detector'])/d['col_monitor'], xerr=None,\
                        #marker='o', ms=5)

    except:
        pass


  1                                scanrel s1 1 -1 0.1 | 4.511   0.001   0.004 | 5.116   0.000  -0.000  -0.000 | [NSF PZ] floff, guide 18 perpq
  2                                scan sgu 0 -3.5 0.5 | 4.511   0.001   0.004 | 5.116   0.000  -0.000  -0.000 | [NSF PZ] floff, guide 18 perpq
  3                                     th2th 2 -2 0.2 | 4.511   0.001   0.004 | 5.116   0.000   0.001  -0.000 | [NSF PZ] floff, guide 18 perpq
  4                                scanrel s1 1 -1 0.1 | 4.511   0.001   0.004 | 5.116   0.000  -0.000  -0.000 | [NSF PZ] floff, guide 18 perpq
  5              scan slit_pre_tp 0 24 2 preset time 1 | 4.511   0.001   0.004 | 5.116  -0.000  -0.000  -0.000 | [NSF PZ] floff, guide 18 perpq
  6              scan slit_pre_bt 0 24 2 preset time 1 | 4.511   0.001   0.004 | 5.116   0.000   0.000  -0.000 | [NSF PZ] floff, guide 18 perpq
  7             scan slit_pre_tp 26 30 2 preset time 1 | 4.511   0.001   0.004 | 5.116   0.000   0.000   0.000 | [NSF PZ] floff, guide 1

In [319]:
d['meta']

{'instrument': 'TAIPAN',
 'filenumber': 193,
 'date': '4/23/2024',
 'time': 5.0,
 'proposal': '32165',
 'experiment': '939',
 'experiment_number': '939',
 'command': 'scanrel caen1 0 0.12 0.003',
 'builtin_command': 'scan caen1 @(caen1)+0 @(caen1)+0.12 0.003000',
 'users': 'Fankang Li, Mason Klemm, Yaofeng Xie',
 'local_contact': 'Masa Matsuda',
 'subtitle': 'CAEN power supply stability test at 10A trial: 9',
 'monochromator': 'Heusler',
 'analyzer': 'Heusler',
 'sense': '+-+',
 'collimation': '48-80-60-999',
 'samplename': 'FeGe',
 'sampletype': 'crystal',
 'samplemosaic': '30.000000',
 'latticeconstants': '5.000000,5.000000,5.000000,90.000000,90.000000,90.000000',
 'ubmatrix': '0.141421,0.141421,-0.000000,-0.000000,-0.000000,0.200000,0.141421,-0.141421,0.000000',
 'mode': '0',
 'plane_normal': '0.000000,0.000000,1.000000',
 'ubconf': 'UB22Apr2024_103013AM.ini',
 'preset_type': 'normal',
 'preset_channel': 'time',
 'preset_value': '5.000000',
 'def_x': 'caen1',
 'def_y': 'detector',
 

In [277]:
np.average(d['col_monitor'])

861.8095238095239